In [20]:
# Usual Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')
import os
from pathlib import Path

In [21]:
current_path = os.getcwd()
current_path = os.getcwd()
genre_music_path = os.path.join(current_path, 'genres_original')
class_xls_path = os.path.join(current_path, 'class_xls')
print(list(os.listdir(genre_music_path)))

['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [22]:
# Importing 1 file
audio_file = os.path.join(genre_music_path,'reggae','reggae.00036.wav')
y, sr = librosa.load(audio_file)

print('y:', y, '\n')
print('y shape:', np.shape(y), '\n')
print('Sample Rate (KHz):', sr, '\n')

# Verify length of the audio
print('Check Len of Audio:', 661794/22050)

y: [0.02072144 0.04492188 0.05422974 ... 0.06912231 0.08303833 0.08572388] 

y shape: (661794,) 

Sample Rate (KHz): 22050 

Check Len of Audio: 30.013333333333332


In [23]:
from codeFeature.feature_extractor import *
import re
from sklearn.model_selection import train_test_split

In [24]:
def GetGenre(genre_music_path):
    label_names = [item for item in os.listdir(
        genre_music_path) if os.path.isdir(os.path.join(genre_music_path, item))]
    nb_train_samples = sum([len(files) for _, _, files in os.walk(genre_music_path)])
    
    return label_names, nb_train_samples
genres, nb_train_samples = GetGenre(genre_music_path)

In [25]:
dir_trainfolder = "./gtzan/_train"
dir_devfolder = "./gtzan/_validation"
dir_testfolder = "./gtzan/_test"
dir_all_files = "./gtzan"

train_X_preprocessed_data = "./gtzan/data_train_input.npy"
train_Y_preprocessed_data = "./gtzan/data_train_target.npy"
dev_X_preprocessed_data = "./gtzan/data_validation_input.npy"
dev_Y_preprocessed_data = "./gtzan/data_validation_target.npy"
test_X_preprocessed_data = "./gtzan/data_test_input.npy"
test_Y_preprocessed_data = "./gtzan/data_test_target.npy"

train_X = train_Y = None
dev_X = dev_Y = None
test_X = test_Y = None

In [26]:
hop_length = 512

timeseries_length_list = []

all_files_list = []

# compute minimum timeseries length, slow to compute, caching pre-computed value of 1290
# precompute_min_timeseries_len()
# print("min(timeseries_length_list) ==" + str(min(timeseries_length_list)))
# timeseries_length = min(timeseries_length_list)

timeseries_length = (
    128
)   # sequence length == 128, default fftsize == 2048 & hop == 512 @ SR of 22050
#  equals 128 overlapped windows that cover approx ~3.065 seconds of audio, which is a bit small!

In [27]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [28]:
def extract_audio_features(list_of_audiofiles):

    data = np.zeros(
        (len(list_of_audiofiles), timeseries_length, 33), dtype=np.float64
    )
    target = []

    for i, file in enumerate(list_of_audiofiles):
        file_name = path_leaf(file)
        
        splits = re.split("[ .]", file_name)
        # print(splits)
        # genre = re.split("[ /]", splits[1])[3]
        genre = splits[0]
        target.append(genre)
        if FileCheck(file)!=1:
            continue
        y, sr = librosa.load(file)
        mfcc = librosa.feature.mfcc(
            y=y, sr=sr, hop_length=hop_length, n_mfcc=13
        )
        spectral_center = librosa.feature.spectral_centroid(
            y=y, sr=sr, hop_length=hop_length
        )
        chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length)
        spectral_contrast = librosa.feature.spectral_contrast(
            y=y, sr=sr, hop_length=hop_length
        )

        data[i, :, 0:13] = mfcc.T[0:timeseries_length, :]
        data[i, :, 13:14] = spectral_center.T[0:timeseries_length, :]
        data[i, :, 14:26] = chroma.T[0:timeseries_length, :]
        data[i, :, 26:33] = spectral_contrast.T[0:timeseries_length, :]

        # print(
        #     "Extracted features audio track %i of %i."
        #     % (i + 1, len(list_of_audiofiles))
        # )

    return data, np.expand_dims(np.asarray(target), axis=1)

In [29]:
def FileCheck(fn):
    try:
        librosa.load(fn, mono=True, duration=30)
        return 1
    except:
        print (f"Error: File {fn} does not appear to exist.")
        return 0

In [30]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [31]:
# [files for _, _, files in os.walk(genre_music_path)]
list_of_audiofiles = [files for files in getListOfFiles(genre_music_path)]


In [32]:
data = extract_audio_features(list_of_audiofiles)

Error: File d:\testRGB\MusicClasiffication\genres_original\jazz\jazz.00054.wav does not appear to exist.


In [33]:
X, y = data

In [34]:
X = np.array(X, dtype = float)

In [35]:
X.shape

(1000, 128, 33)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

In [37]:
X_train.shape

(750, 128, 33)

In [38]:
input_shape = (X_train.shape[1], X_train.shape[2])

In [39]:
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import model_from_json

In [40]:
model = Sequential()
model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=input_shape))
model.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(units=len(genres), activation="softmax"))

In [41]:
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# model.summary()

In [42]:
X_train.shape

(750, 128, 33)

In [43]:
def one_hot(Y_genre_strings):
        y_one_hot = np.zeros((Y_genre_strings.shape[0], len(genres)))
        for i, genre_string in enumerate(Y_genre_strings):
            index = genres.index(genre_string)
            y_one_hot[i, index] = 1
        return y_one_hot

In [44]:
ytest_train = one_hot(y_train)

In [45]:
ytes_test = one_hot(y_test)

In [49]:
batch_size = 100  # num of training examples per minibatch
num_epochs = 100
model.fit(
    X_train,
    ytest_train,
    validation_data=(X_test, ytes_test),
    batch_size=batch_size,
    epochs=num_epochs,
)

Epoch 1/100
8/8 [==============================] - 14s 2s/step - loss: 0.5120 - accuracy: 0.8093 - val_loss: 1.8931 - val_accuracy: 0.4240
Epoch 2/100
8/8 [==============================] - 13s 2s/step - loss: 0.4889 - accuracy: 0.8307 - val_loss: 2.0067 - val_accuracy: 0.4360
Epoch 3/100
8/8 [==============================] - 13s 2s/step - loss: 0.4949 - accuracy: 0.8267 - val_loss: 1.9444 - val_accuracy: 0.4320
Epoch 4/100
8/8 [==============================] - 13s 2s/step - loss: 0.5133 - accuracy: 0.8133 - val_loss: 1.8912 - val_accuracy: 0.4320
Epoch 5/100
8/8 [==============================] - 13s 2s/step - loss: 0.4942 - accuracy: 0.8293 - val_loss: 1.8854 - val_accuracy: 0.4840
Epoch 6/100
8/8 [==============================] - 13s 2s/step - loss: 0.4754 - accuracy: 0.8347 - val_loss: 1.8405 - val_accuracy: 0.4680
Epoch 7/100
8/8 [==============================] - 14s 2s/step - loss: 0.5028 - accuracy: 0.8440 - val_loss: 1.9606 - val_accuracy: 0.4400
Epoch 8/100
8/8 [==========

In [50]:
# Creates a HDF5 file 'lstm_genre_classifier.h5'
model_filename = "./gtzan/model_weights.h5"
print("\nSaving model: " + model_filename)
model.save(model_filename)
# Creates a json file
# print("creating .json file....")
model_json = model.to_json()
f = "./gtzan/model.json"
#save the model architecture to JSON file
with open(f, 'w') as json_file:
    json_file.write(model_json)



Saving model: ./gtzan/model_weights.h5


In [63]:
# X_train, X_test, y_train, y_test 

In [51]:
train_X_preprocessed_data = "./gtzan/data_train_input.npy"
train_Y_preprocessed_data = "./gtzan/data_train_target.npy"
dev_X_preprocessed_data = "./gtzan/data_validation_input.npy"
dev_Y_preprocessed_data = "./gtzan/data_validation_target.npy"
test_X_preprocessed_data = "./gtzan/data_test_input.npy"
test_Y_preprocessed_data = "./gtzan/data_test_target.npy"

In [52]:
with open(train_X_preprocessed_data, "wb") as f:
            np.save(f, X_train)
with open(train_Y_preprocessed_data, "wb") as f:
            np.save(f, one_hot(y_train))

In [53]:
with open(test_X_preprocessed_data, "wb") as f:
            np.save(f, X_test)
with open(test_Y_preprocessed_data, "wb") as f:
            np.save(f, one_hot(y_test))

In [54]:
print("\nValidating ...")
score, accuracy = model.evaluate(
    X_test, one_hot(y_test), batch_size=batch_size, verbose=1
)
print("Dev loss:  ", score)
print("Dev accuracy:  ", accuracy)


Validating ...
3/3 [==============================] - 1s 242ms/step - loss: 1.9772 - accuracy: 0.4400
Dev loss:   1.9771649837493896
Dev accuracy:   0.4399999976158142


In [55]:
def load_model(model_path, weights_path):
    "Load the trained LSTM model from directory for genre classification"
    with open(model_path, "r") as model_file:
        trained_model = model_from_json(model_file.read())
    trained_model.load_weights(weights_path)
    trained_model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return trained_model


def extract_audio_features(file):
    "Extract audio features from an audio file for genre classification"
    timeseries_length = 128
    features = np.zeros((1, timeseries_length, 33), dtype=np.float64)

    y, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512, n_mfcc=13)
    spectral_center = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=512)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=512)

    features[0, :, 0:13] = mfcc.T[0:timeseries_length, :]
    features[0, :, 13:14] = spectral_center.T[0:timeseries_length, :]
    features[0, :, 14:26] = chroma.T[0:timeseries_length, :]
    features[0, :, 26:33] = spectral_contrast.T[0:timeseries_length, :]
    return features


def get_genre(model, music_path):
    "Predict genre of music using a trained model"
    prediction = model.predict(extract_audio_features(music_path))
    predict_genre = genres[np.argmax(prediction)]
    return predict_genre


In [62]:
if __name__ == "__main__":
    PATH = "./genres_original/reggae/reggae.00015.wav"
    MODEL = load_model("./gtzan/model.json", "./gtzan/model_weights.h5")
    GENRE = get_genre(MODEL, PATH)
    print("Model predict: {}".format(GENRE))

Model predict: reggae
